<a href="https://colab.research.google.com/github/MenaHagag/Capstone-Project-/blob/master/City_of_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

In [10]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup # beautifulsoup
import requests
print('Libraries imported.')

Libraries imported.


In [0]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text



In [0]:
# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
print(soup.prettify())

In [0]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

 #print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        if (Postcode_var == 'Not assigned\n' or Borough_var == 'Not assigned\n' or Neighbourhood_var == 'Not assigned\n'):
            continue
        if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
            continue
        
        Postcode.append(Postcode_var[:-1])
        Borough.append(Borough_var[:-1])
        Neighbourhood.append(Neighbourhood_var)
    """ if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue 
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
    print(Postcode_var)
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var) """

In [0]:
Borough

In [67]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 103


In [71]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(11)

,Postcode,Borough,Neighbourhood
0,M4C,East York,Woodbine Heights
1,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
2,M9W,Etobicoke,Northwest
3,M5H,Downtown Toronto,"Richmond, Adelaide, King"
4,M3A,North York,Parkwoods
5,M4B,East York,"Parkview Hill, Woodbine Gardens"
6,M2N,North York,Willowdale
7,M1E,Scarborough,"Guildwood, Morningside, West Hill"
8,M1N,Scarborough,"Birch Cliff, Cliffside West"
9,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"


In [73]:
df_toronto.shape

(103, 3)